In [1]:
##--------步骤1----------导入相关包，数据
import sys
!{sys.executable} -m pip install pandasql

!{sys.executable} -m pip install pysqldf

##pip install -U pandasql
##pip install pandasql
%matplotlib inline
##每行显示
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
###from pandasql import sqldf
##train
##tt1=pd.merge(waybill_info, area_realtime, on='area_id', how='left')
##tt1.head(10)
from pysqldf import SQLDF, load_meat, load_births
sqlfun = SQLDF(globals())


area_realtime=pd.read_csv('./area_realtime.csv');
waybill_info=pd.read_csv('./waybill_info.csv');
weather_realtime=pd.read_csv('./weather_realtime.csv');
##area_realtime.dtypes
##area_realtime.head()

##test
##area_realtime_test=pd.read_csv('./area_realtime_test.csv');
##waybill_info_test_a=pd.read_csv('./waybill_info_test_a.csv');
#waybill_info_test_b=pd.read_csv('./waybill_info_test_b.csv');
#weather_realtime_test=pd.read_csv('./weather_realtime_test.csv');
##subm
sample_submission=pd.read_csv('./sample_submission.csv');

area_realtime.head(1)
print area_realtime.dtypes
print area_realtime.shape

  Running setup.py bdist_wheel for pandasql ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/a4/4b/e6/88a009d13e4b64f7292e802a93d19619cceb135ba599235430
Successfully built pandasql


/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


date                         int64
time                         int64
log_unix_time                int64
area_id                      int64
working_rider_num            int64
notbusy_working_rider_num    int64
not_fetched_order_num        int64
deliverying_order_num        int64
dtype: object
(443520, 8)


In [ ]:
##--------步骤2-----将订单数据表关联位置表，并保存文件
qq="""
select 
 t1.order_id
,t1.poi_id
,t1.area_id
,t1.food_total_value
,t1.box_total_value
,t1.food_num
,t1.delivery_distance
,t1.order_unix_time
,t1.customer_longitude
,t1.customer_latitude
,t1.poi_lng
,t1.poi_lat
,t1.waiting_order_num
,t1.delivery_duration
,t1.fetch_unix_time-t2.log_unix_time as difftim
,t2.date
,t2.time
,t2.log_unix_time  as  log_unix_time2
,t2.working_rider_num
,t2.notbusy_working_rider_num
,t2.not_fetched_order_num
,t2.deliverying_order_num
from  waybill_info t1
left join area_realtime t2 
on t1.area_id=t2.area_id
and  t1.fetch_unix_time-t2.log_unix_time<=30
and   t1.fetch_unix_time-t2.log_unix_time>=-30
where t2.area_id is not null
and  t1.fetch_unix_time-t2.log_unix_time<=30
and   t1.fetch_unix_time-t2.log_unix_time>=-30
;
""" 
train1_full=sqlfun.execute(qq) 
train1_full.to_csv('./train1_full.csv')
train1_full=pd.read_csv('./train1_full.csv');
train1_full.head(10)


,Unnamed: 0,order_id,poi_id,area_id,food_total_value,box_total_value,food_num,delivery_distance,order_unix_time,customer_longitude,...,waiting_order_num,delivery_duration,difftim,date,time,log_unix_time2,working_rider_num,notbusy_working_rider_num,not_fetched_order_num,deliverying_order_num
0,0,1499011202100945,3025506,1003209,43.0,3.0,NaN,2935.0,1499011015,163.388616,...,0.0,1780.0,-23.0,20170703,12,1499011921,6,5,0,1
1,1,1499011258143975,3438720,1003189,73.0,2.0,NaN,483.0,1499011064,163.619991,...,0.0,2057.0,13.0,20170703,27,1499012821,9,9,6,2
2,2,1499011404020428,1320682,1002570,38.9,0.0,2.0,630.0,1499011369,163.428880,...,0.0,854.0,30.0,20170703,10,1499011801,9,8,3,0
3,3,1499011404020428,1320682,1002570,38.9,0.0,2.0,630.0,1499011369,163.428880,...,0.0,854.0,-30.0,20170703,11,1499011861,9,8,2,1
4,4,1499011547078180,3204726,1002483,65.0,3.0,1.0,922.0,1499011456,163.566002,...,0.0,1745.0,4.0,20170703,28,1499012881,4,4,5,0
5,5,1499011654054912,1011359,1002483,41.0,1.0,1.0,3671.0,1499011630,163.562448,...,0.0,2584.0,27.0,20170703,29,1499012941,4,4,4,1
6,6,1499011661133501,1622041,1002570,76.0,7.0,1.0,1911.0,1499011648,163.411028,...,0.0,1349.0,-23.0,20170703,21,1499012461,9,8,3,2
7,7,1499011764108405,1587154,1002570,53.0,3.0,1.0,787.0,1499011730,163.413603,...,0.0,1065.0,-7.0,20170703,24,1499012641,9,8,1,4
8,8,1499011828131475,3127969,1004285,70.0,4.0,1.0,2519.0,1499011819,163.655112,...,0.0,1562.0,9.0,20170703,26,1499012761,3,3,3,3
9,9,1499011881169038,1492360,1002570,56.0,2.0,1.0,2027.0,1499011872,163.418192,...,0.0,1496.0,-27.0,20170703,26,1499012761,9,8,0,5


In [2]:
##
train3=pd.read_csv('./train1_full.csv');
print train3
##1019168条记录

         Unnamed: 0          order_id   poi_id  area_id  food_total_value  \
0                 0  1499011202100945  3025506  1003209              43.0   
1                 1  1499011258143975  3438720  1003189              73.0   
2                 2  1499011404020428  1320682  1002570              38.9   
3                 3  1499011404020428  1320682  1002570              38.9   
4                 4  1499011547078180  3204726  1002483              65.0   
5                 5  1499011654054912  1011359  1002483              41.0   
6                 6  1499011661133501  1622041  1002570              76.0   
7                 7  1499011764108405  1587154  1002570              53.0   
8                 8  1499011828131475  3127969  1004285              70.0   
9                 9  1499011881169038  1492360  1002570              56.0   
10               10  1499011988116871  1276552  1003189              40.0   
11               11  1499011961043502  3127969  1004285              28.0   

In [2]:
##----------------步骤3--------训练集与测试集数据预处理，包括填充缺失值，正太归一化
train3=pd.read_csv('./train1_full.csv');
train3.head(20)
X_train_without_temp=train3[[ 'food_total_value',
       'box_total_value', 'food_num', 'delivery_distance',
       'poi_lng', 'poi_lat', 'waiting_order_num',
       'time', 'working_rider_num',
       'notbusy_working_rider_num', 'not_fetched_order_num',
       'deliverying_order_num']]
mean_cols = X_train_without_temp.mean()
##print mean_cols
X_train_without_temp = X_train_without_temp.fillna(mean_cols)

numeric_cols = X_train_without_temp.columns[X_train_without_temp.dtypes != 'object']
#print (numeric_cols)


numeric_col_means = X_train_without_temp.loc[:,numeric_cols].mean()
numeric_col_std = X_train_without_temp.loc[:,numeric_cols].std()
X_train_without_temp.loc[:,numeric_cols] = (X_train_without_temp.loc[:,numeric_cols] - numeric_col_means) / numeric_col_std

print 'Train date \n ',   type(X_train_without_temp),'\n',X_train_without_temp.head()
X_train_without_temp.to_csv('./X_train_without_temp.csv')
##X_train_without_temp=pd.read_csv('./X_train_without_temp.csv');
#print 'mean'
#print numeric_col_means
#print 'std'
#print numeric_col_std
#print 'res'
#print X_train_without_temp
####y预处理
y_traintmp=train3[[ 'delivery_duration']]
y_traintmp = y_traintmp.fillna(y_traintmp.mean())
y_train=np.log1p(y_traintmp)
print 'Train Y \n ', type(y_train),'\n',  y_train.head()


#测试集预处理
test3=pd.read_csv('./test1.csv');
test3.head(20)
order_id=test3[["order_id"]]
X_test_without_temp=test3[[ 'food_total_value',
       'box_total_value', 'food_num', 'delivery_distance',
       'poi_lng', 'poi_lat', 'waiting_order_num',
       'time', 'working_rider_num',
       'notbusy_working_rider_num', 'not_fetched_order_num',
       'deliverying_order_num']]

X_test_without_temp = X_test_without_temp.fillna(mean_cols)
X_test_without_temp.loc[:,numeric_cols] = (X_test_without_temp.loc[:,numeric_cols] - numeric_col_means) / numeric_col_std

print 'test data\n ' ,  type(X_test_without_temp),'\n',X_test_without_temp.head()
X_test_without_temp.to_csv('./X_test_without_temp.csv')



Train date 
  <class 'pandas.core.frame.DataFrame'> 
   food_total_value  box_total_value      food_num  delivery_distance  \
0         -0.430641         0.316072  3.301068e-14           1.453280   
1          0.225053        -0.130612  3.301068e-14          -1.348644   
2         -0.520253        -1.023982  1.093723e+00          -1.180666   
3         -0.520253        -1.023982  1.093723e+00          -1.180666   
4          0.050201         0.316072 -1.878883e-01          -0.846995   

    poi_lng   poi_lat  waiting_order_num      time  working_rider_num  \
0 -1.200863 -1.295609          -0.536906 -3.897415          -1.459075   
1  1.114622 -1.397497          -0.536906 -3.857727          -1.420948   
2 -0.795590  1.982019          -0.536906 -3.902706          -1.420948   
3 -0.795590  1.982019          -0.536906 -3.900060          -1.420948   
4  0.509535  0.814632          -0.536906 -3.855081          -1.484493   

   notbusy_working_rider_num  not_fetched_order_num  deliverying_orde

In [16]:

###------------------------步骤4----------训练
import time
from xgboost import XGBRegressor
X_train=X_train_without_temp
y_train=y_train
##初始化模块
xgb = XGBRegressor(max_depth =50)
##计时
time_start=time.time()
##开始训练
xgb.fit(X_train, y_train)

time_end=time.time()

##xgb.save_model('xgb.model')

print('totally time',time_end-time_start)
print xgb


('totally time', 573.5502829551697)
XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=50,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)


In [ ]:
# load model and data in
from xgboost import XGBRegressor
 import  xgboost  as  xbt
xgb = xgb.Booster(model_file='xgb.model')

In [ ]:
####-----------步骤5---------------预测
from pandas import Series, DataFrame 
import pandas as pd
X_test=X_test_without_temp
##print X_test
y_finaltmp = xgb.predict(X_test)
y_final=np.expm1(y_finaltmp)
##y_=np.transpose(y_final)
y_=y_final.reshape(y_final.shape[0],1)
result=pd.DataFrame(y_ ,columns=["delivery_duration"])

order_id=test3[["order_id"]]
result2 = pd.concat([order_id,result], axis=1)
result3=result2[['order_id','delivery_duration']]
result3.to_csv('./submission.csv',index=False)
result3=pd.read_csv('./submission.csv');
print result3
##print type(result)
##拿训练集验证一下



In [8]:
####-----------步骤6---------------去重
from pandas import Series, DataFrame 
import pandas as pd


result=pd.read_csv('./result_2.csv');
result2=result[['order_id','delivery_duration']]

qq="""
    select 
           order_id
         , (max(delivery_duration)+min(delivery_duration))/2.0 as delivery_duration
      from result2
     group by order_id
"""

result3=sqlfun.execute(qq) 
##print sqlfun.execute("select count(*) from  result3") 
##250104

result3.to_csv('./submission1.csv',index=False)
#print result3
##补充订单
order_id_full=sqlfun.execute("select distinct order_id  from  sample_submission") 
##mean_duration = result2[['delivery_duration']].mean()

##print 'mean_duration',mean_duration
##1879.842762
qq="""
select 
   t2.order_id
 , case when  t3.order_id is not null then t3.delivery_duration 
        when  t3.order_id is  null  then  1879.842762
        else  0
   end  as delivery_duration
 from order_id_full  t2 
 left join  result3  t3
   on t2.order_id=t3=order_id
"""

qq2="""
select 
   t2.order_id
 , t3. delivery_duration
 from order_id_full  t2 
 left join  result3  t3
   on t2.order_id=t3.order_id
"""
result4=sqlfun.execute(qq2) 
result5 = result4.fillna(1879.842762)
result5.to_csv('./submission4.csv',index=False)

##print type(result)

print sqlfun.execute("select count(*) from  result5") 
##拿训练集验证一下

   count(*)
0    251864


In [2]:
order_id_full=sqlfun.execute("select count(distinct order_id ) from  sample_submission") 

qq="""
select 
   t2.order_id
 , case when t3.order_id is not null then t3.delivery_duration 
        when t3.order_id is  null  then  1879.842762
   end as delivery_duration
 from order_id_full  t2 
 left join  result3  t3
   on t2.order_id=t3=order_id
"""

/opt/conda/envs/python2/lib/python2.7/site-packages/pandas/io/sql.py:525: FutureWarning: the 'flavor' parameter is deprecated and will be removed in a future version, as 'sqlite' is the only supported option when SQLAlchemy is not installed.
  _validate_flavor_parameter(flavor)


   count(distinct order_id )
0                     251864


##结束